Реализована явная схема Адамса для решения задачи Коши с двумя уравнениями первого порядка ($u'=f(t, u),\:u(0)=u(t_0)=u_0$): $u_{k+1}=u_k + \tau (\frac{3}{2}f_k - \frac{1}{2}f_{k-1}),\:k = 1,..., n - 1$, где $\tau=t_{i+1} - t_i,\:f_k=f(t_k, u_k),\:u_k=u(t_k)$. $t_k:\:k=0,...,n,\:t_0=0,\:t_n=T$. Для этой схемы требуется сначала найти $u_1$, для этого используется явная схема Эйлера: $u_1 = u_0 + \tau f(t_0, u_0)$.

In [19]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

def Koshi_Adams(t_, f_0, f_1, u_0):
    '''
    t_ - значения t_i
    f_0, f_1 - правые части первого и второго уравнений соответсвенно 
    (функции, принимающие одно число t и массив из двух значений u)
    u_0 - начальные условия
    '''
    t = np.array(t_)
    n = t.shape[0]
    tao = t[1] - t[0]
    u = np.empty((2, n))
    u[:, 0] = np.reshape(u_0, (2))
    f0 = np.array([f_0(t[0], u_0), f_1(t[0], u_0)])
    f1 = np.empty(2)
    # вычислим u_1 с помощью явной схемы Эйлера
    u[:, 1] = u[:, 0] + tao * f0
    # вычислим остальные значения u_k с помощью явной схемы Адамса
    for k in range(1, n - 1):
        f1 = np.array([f_0(t[k], u[:, k]), f_1(t[k], u[:, k])])
        u[:, k + 1] = u[:, k] + (tao / 2) * (3 * f1 - f0)
        f0, f1 = f1, f0
    return u